In [ ]:
import random
import numpy as np
import PIL
import PIL.ImageOps
import PIL.ImageEnhance
import PIL.ImageDraw
from PIL import Image

PARAMETER_MAX = 10
TRANSLATE_CONST = 22  # Used to scale translation arguments


# Augmentation operations
def AutoContrast(img, **kwargs):
    return PIL.ImageOps.autocontrast(img)


def Brightness(img, level, bias=0.8):
    level = level * 0.4 + bias  # +-20%
    return PIL.ImageEnhance.Brightness(img).enhance(level)


# def Color(img, level, bias=0.1):
#     level = level * 1.8 + bias  # Scaling and bias as per TensorFlow
#     return PIL.ImageEnhance.Color(img).enhance(level)


def Contrast(img, level, bias=0.8):
    level = level * 0.4 + bias  # +-20%
    return PIL.ImageEnhance.Contrast(img).enhance(level)


def Cutout(img, level, **kwargs):
    if level == 0:
        return img
    level = int(level * min(img.size))
    return CutoutAbs(img, level)


def CutoutAbs(img, level, **kwargs):
    w, h = img.size
    x0 = np.random.uniform(0, w)
    y0 = np.random.uniform(0, h)
    x0 = int(max(0, x0 - level / 2.))
    y0 = int(max(0, y0 - level / 2.))
    x1 = int(min(w, x0 + level))
    y1 = int(min(h, y0 + level))
    xy = (x0, y0, x1, y1)
    color = (127, 127, 127)  # Gray color for cutout
    img = img.copy()
    PIL.ImageDraw.Draw(img).rectangle(xy, color)
    return img


def Equalize(img, **kwargs):
    return PIL.ImageOps.equalize(img)


def Identity(img, **kwargs):
    return img


def Rotate(img, level, bias=0):
    level = level * 20
    if random.random() < 0.5:  # Randomly flip direction
        level = -level
    return img.rotate(level)


def ShearX(img, level, bias=0):
    level = level * 0.1
    if random.random() < 0.5:  # Randomly flip direction
        level = -level
    return img.transform(img.size, PIL.Image.AFFINE, (1, level, 0, 0, 1, 0))


def ShearY(img, level, bias=0):
    level = level * 0.1
    if random.random() < 0.5:  # Randomly flip direction
        level = -level
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, level, 1, 0))


def TranslateX(img, level, bias=0):
    level = level * TRANSLATE_CONST / PARAMETER_MAX  # Scaling as per TensorFlow
    if random.random() < 0.5:  # Randomly flip direction
        level = -level
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, level, 0, 1, 0))


def TranslateY(img, level, bias=0):
    level = level * TRANSLATE_CONST / PARAMETER_MAX  # Scaling as per TensorFlow
    if random.random() < 0.5:  # Randomly flip direction
        level = -level
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, 0, 1, level))


# Augmentation pool for FixMatch
def fixmatch_augment_pool():
    return [
        (AutoContrast, None),
        (Brightness, 0.8),  # Correct bias for Brightness
        # (Color, 0.1),       # Correct bias for Color
        (Contrast, 0.8),    # Correct bias for Contrast
        (Equalize, None),
        (Identity, None),
        # (Rotate, 0),
        (ShearX, 0),
        (ShearY, 0),
        (TranslateX, 0),
        (TranslateY, 0),
    ]


# RandAugmentMC aligned with FixMatch
class RandAugmentMC:
    def __init__(self, n, m, prob_to_apply=0.5, level=None, num_levels=None):
        """
        :param n: Number of augmentations to apply
        :param m: Maximum augmentation magnitude
        :param prob_to_apply: Probability of applying a selected augmentation
        :param level: Fixed augmentation level (if specified)
        :param num_levels: Number of discrete levels for random level selection
        """
        assert n >= 1
        assert 1 <= m <= PARAMETER_MAX
        self.n = n
        self.m = m
        self.prob_to_apply = prob_to_apply
        self.level = level
        self.num_levels = num_levels
        self.augment_pool = fixmatch_augment_pool()

    def _get_level(self):
        """Determine the augmentation level."""
        if self.level is not None:
            return self.level  # Use fixed level
        elif self.num_levels is not None:
            return random.randint(0, self.num_levels) / self.num_levels  # Discrete random level
        else:
            return random.uniform(0, 1)  # Uniform random level

    def __call__(self, img):
        ops = random.choices(self.augment_pool, k=self.n)  # Select n augmentations
        for op, bias in ops:
            if random.random() < self.prob_to_apply:  # Apply augmentation with specified probability
                level = self._get_level() * self.m
                img = op(img, level=level, bias=bias)
        img = CutoutAbs(img, int(self.m / PARAMETER_MAX * min(img.size)))  # Apply scaled Cutout
        return img


In [ ]:
# m = 0.5-1 - Determined by cutout since m/10 should be 5-10%
# Shear 0.1
# Rotation 20
#Image size after crop and resize = 224
# Translation < 5-10% : (level/10)*(t/224); t = 22

